Melhorias adicionadas:

    Adicionado comentários em pt-br para explicar o código
    Utilizado TrainingArguments para definir argumentos de treinamento
    Utilizado Trainer para treinar o modelo
    Adicionado compute_metrics para calcular métricas durante o treinamento
    Utilizado pipeline para criar uma pipeline de geração de texto
    Adicionado num_return_sequences para especificar o número de sequências a serem geradas

Essas são apenas algumas sugestões de melhorias, e há muitas outras coisas que você pode fazer para melhorar o código.

In [2]:
# Importar bibliotecas necessárias
from datasets import load_dataset
from transformers import (
    GPT2Tokenizer,
    GPT2Config,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    pipeline
)

/home/alysson/workspace/Criando-um-LLM-modelo-de-linguagem-de-grande-escala-do-zero-com-Transformers/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Carregar dataset
dataset = load_dataset("wikitext", "wikitext-103-v1")
train_data = dataset['train']['text']

In [ ]:
# Criar tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Codificar dados
train_encodings = tokenizer(
    train_data,
    return_tensors='pt',
    max_length=1,
    truncation=True,
    padding='max_length'
)

In [ ]:
# Criar configuração do modelo
config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_positions=512,
    n_ctx=512,
    n_embd=768,
    n_layer=12,
    n_head=12
)


In [ ]:
# Criar modelo
model = GPT2LMHeadModel(config)

In [ ]:
# Criar argumentos de treinamento
training_args = TrainingArguments(
    output_dir='./meuModeloGPT2',  
    num_train_epochs=5,  
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,  
    evaluation_strategy='epoch',  
    learning_rate=5e-5,  
    save_total_limit=2,  
    save_strategy='epoch',  # Changed from 'teps' to 'epoch'
    load_best_model_at_end=True,  
    metric_for_best_model='loss',  
    greater_is_better=False,  
    eval_accumulation_steps=10,  
)

NameError: name 'TrainingArguments' is not defined

In [ ]:
# Criar treinador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
    eval_dataset=train_encodings,
    compute_metrics=lambda pred: {'loss': pred.loss}  # Função para calcular métricas
)

In [ ]:
# Treinar modelo
trainer.train()

In [ ]:
# Criar pipeline de geração de texto
generator = pipeline('text-generation', model='./meuModeloGPT2', tokenizer='gpt2')

In [ ]:
# Gerar texto
prompt = "Qual é o significado da vida, do universo e tudo mais?"
response = generator(prompt, max_length=50, num_return_sequences=1)
print(response[0]['generated_text'])  # Imprimir texto gerado
